In [10]:
import os

os.environ["PHONEMIZER_ESPEAK_LIBRARY"] = "/usr/bin/espeak"

import phonemizer
phonemizer.phonemize("你好")

'tʃaɪniːzlɛɾɚ lɛɾɚ '

## MB-iSTFT-VITS2 inference

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa

import os
import json
import math
import numpy as np

import requests
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence
import langdetect

from scipy.io.wavfile import write
import re

In [18]:
from bert_extract import VITS_BERT

vits_bert = VITS_BERT("models/bert-prosody/", "cuda")

def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm


def langdetector(text):  # from PolyLangVITS
    try:
        lang = langdetect.detect(text).lower()
        if lang == 'ko':
            return f'[KO]{text}[KO]'
        elif lang == 'ja':
            return f'[JA]{text}[JA]'
        elif lang == 'en':
            return f'[EN]{text}[EN]'
        elif lang == 'zh-cn':
            return f'[ZH]{text}[ZH]'
        else:
            return text
    except Exception as e:
        return text


def vcss(inputstr, audiopath):
    fltstr = re.sub(r"[\[\]\(\)\{\}]", "", inputstr)
    #fltstr = langdetector(fltstr)
    stn_tst = get_text(fltstr, hps)
    print(stn_tst)

    speed = 1
    sid = 0
    output_dir = 'output'
    with torch.no_grad():
        x_tst = stn_tst.cpu().unsqueeze(0)
        x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cpu()
        emo = torch.FloatTensor(np.load(audiopath+".emo.npy")).cpu().unsqueeze(0)
        bertpt = torch.FloatTensor(vits_bert.chinese_to_bert(fltstr).T).cuda().unsqueeze(0)
        audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, emo=emo, bertpt=bertpt,noise_scale=.667, noise_scale_w=0.8, length_scale=1 / speed)[0][
                0, 0].data.cpu().float().numpy()

    ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

In [19]:
# hps = utils.get_hparams_from_file("./configs/mini_mb_istft_vits2_base.json")
hps = utils.get_hparams_from_file("./configs/mb_istft_vits2_base_esd_aishell3.json")

if "use_mel_posterior_encoder" in hps.model.keys() and hps.model.use_mel_posterior_encoder == True:
    print("Using mel posterior encoder for VITS2")
    posterior_channels = 80  # vits2
    hps.data.use_mel_posterior_encoder = True
else:
    print("Using lin posterior encoder for VITS1")
    posterior_channels = hps.data.filter_length // 2 + 1
    hps.data.use_mel_posterior_encoder = False

net_g = SynthesizerTrn(
    len(symbols),
    posterior_channels,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint("./logs/mb_istft_vits2_base/esd_aishell3/G_3000.pth", net_g, None)

Using mel posterior encoder for VITS2
Mutli-band iSTFT VITS2


In [20]:
# - text input
input = "你好"
audiopath = "/data/dataset"
vcss(input, "/data/datasets/ESD/0001/Angry/0001_000351.wav")

tensor([ 0, 13, 41, 46, 17, 34, 46,  0])
[PAD]你好[PAD] [1, 3, 3, 1] 8 4


In [62]:
import os
import re

import cn2an
from pypinyin import lazy_pinyin, Style

from text.symbols import punctuation
from text.tone_sandhi import ToneSandhi

current_file_path = os.path.dirname(__file__)
pinyin_to_symbol_map = {
    line.split("\t")[0]: line.strip().split("\t")[1]
    for line in open(os.path.join(current_file_path, "opencpop-strict.txt")).readlines()
}

import jieba.posseg as psg


rep_map = {
    "：": ",",
    "；": ",",
    "，": ",",
    "。": ".",
    "！": "!",
    "？": "?",
    "\n": ".",
    "·": ",",
    "、": ",",
    "...": "…",
    "$": ".",
    "“": "'",
    "”": "'",
    "‘": "'",
    "’": "'",
    "（": "'",
    "）": "'",
    "(": "'",
    ")": "'",
    "《": "'",
    "》": "'",
    "【": "'",
    "】": "'",
    "[": "'",
    "]": "'",
    "—": "-",
    "～": "-",
    "~": "-",
    "「": "'",
    "」": "'",
}

tone_modifier = ToneSandhi()


def replace_punctuation(text):
    text = text.replace("嗯", "恩").replace("呣", "母")
    pattern = re.compile("|".join(re.escape(p) for p in rep_map.keys()))

    replaced_text = pattern.sub(lambda x: rep_map[x.group()], text)

    replaced_text = re.sub(
        r"[^\u4e00-\u9fa5" + "".join(punctuation) + r"]+", "", replaced_text
    )

    return replaced_text


def g2p(text):
    pattern = r"(?<=[{0}])\s*".format("".join(punctuation))
    sentences = [i for i in re.split(pattern, text) if i.strip() != ""]
    phones, tones, word2ph = _g2p(sentences)
    assert sum(word2ph) == len(phones)
    assert len(word2ph) == len(text)  # Sometimes it will crash,you can add a try-catch.
    phones = ["_"] + phones + ["_"]
    tones = [0] + tones + [0]
    word2ph = [1] + word2ph + [1]
    return phones, tones, word2ph


def _get_initials_finals(word):
    initials = []
    finals = []
    orig_initials = lazy_pinyin(word, neutral_tone_with_five=True, style=Style.INITIALS)
    orig_finals = lazy_pinyin(
        word, neutral_tone_with_five=True, style=Style.FINALS_TONE3
    )
    for c, v in zip(orig_initials, orig_finals):
        initials.append(c)
        finals.append(v)
    return initials, finals


def _g2p(segments):
    phones_list = []
    tones_list = []
    word2ph = []
    for seg in segments:
        # Replace all English words in the sentence
        seg = re.sub("[a-zA-Z]+", "", seg)
        seg_cut = psg.lcut(seg)
        initials = []
        finals = []
        seg_cut = tone_modifier.pre_merge_for_modify(seg_cut)
        for word, pos in seg_cut:
            if pos == "eng":
                continue
            sub_initials, sub_finals = _get_initials_finals(word)
            sub_finals = tone_modifier.modified_tone(word, pos, sub_finals)
            initials.append(sub_initials)
            finals.append(sub_finals)

            # assert len(sub_initials) == len(sub_finals) == len(word)
        initials = sum(initials, [])
        finals = sum(finals, [])
        #
        for c, v in zip(initials, finals):
            raw_pinyin = c + v
            # NOTE: post process for pypinyin outputs
            # we discriminate i, ii and iii
            if c == v:
                assert c in punctuation
                phone = [c]
                tone = "0"
                word2ph.append(1)
            else:
                v_without_tone = v[:-1]
                tone = v[-1]

                pinyin = c + v_without_tone
                assert tone in "12345"

                if c:
                    # 多音节
                    v_rep_map = {
                        "uei": "ui",
                        "iou": "iu",
                        "uen": "un",
                    }
                    if v_without_tone in v_rep_map.keys():
                        pinyin = c + v_rep_map[v_without_tone]
                else:
                    # 单音节
                    pinyin_rep_map = {
                        "ing": "ying",
                        "i": "yi",
                        "in": "yin",
                        "u": "wu",
                    }
                    if pinyin in pinyin_rep_map.keys():
                        pinyin = pinyin_rep_map[pinyin]
                    else:
                        single_rep_map = {
                            "v": "yu",
                            "e": "e",
                            "i": "y",
                            "u": "w",
                        }
                        if pinyin[0] in single_rep_map.keys():
                            pinyin = single_rep_map[pinyin[0]] + pinyin[1:]

                assert pinyin in pinyin_to_symbol_map.keys(), (pinyin, seg, raw_pinyin)
                phone = pinyin_to_symbol_map[pinyin].split(" ")
                word2ph.append(len(phone))

            phones_list += phone
            tones_list += [int(tone)] * len(phone)
    return phones_list, tones_list, word2ph


def text_normalize(text):
    numbers = re.findall(r"\d+(?:\.?\d+)?", text)
    for number in numbers:
        text = text.replace(number, cn2an.an2cn(number), 1)
    text = replace_punctuation(text)
    return text


def get_bert_feature(text, word2ph):
    from text import chinese_bert

    return chinese_bert.get_bert_feature(text, word2ph)


import torch
import sys
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("./bert/chinese-roberta-wwm-ext-large")

models = dict()


def get_bert_feature(text, word2ph, device=None):
    if (
        sys.platform == "darwin"
        and torch.backends.mps.is_available()
        and device == "cpu"
    ):
        device = "mps"
    if not device:
        device = "cuda"
    if device not in models.keys():
        models[device] = AutoModelForMaskedLM.from_pretrained(
            "./bert/chinese-roberta-wwm-ext-large"
        ).to(device)
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors="pt")
        for i in inputs:
            inputs[i] = inputs[i].to(device)
        res = models[device](**inputs, output_hidden_states=True)
        res = torch.cat(res["hidden_states"][-3:-2], -1)[0].cpu()

    assert len(word2ph) == len(text) + 2
    word2phone = word2ph
    phone_level_feature = []
    for i in range(len(word2phone)):
        repeat_feature = res[i].repeat(word2phone[i], 1)
        phone_level_feature.append(repeat_feature)

    phone_level_feature = torch.cat(phone_level_feature, dim=0)

    return phone_level_feature.T



phone_level_feature = torch.cat(phone_level_feature, dim=0)
print(phone_level_feature.shape)  # torch.Size([36, 1024])

text = "啊！但是《原神》是由,米哈\游自主，  [研发]的一款全.新开放世界.冒险游戏"
text = text_normalize(text)
print(text)
phones, tones, word2ph = g2p(text)
bert = get_bert_feature(text, word2ph)

print(phones, tones, word2ph, bert.shape)


# # 示例用法
# text = "这是一个示例文本：,你好！这是一个测试...."
# print(g2p_paddle(text))  # 输出: 这是一个示例文本你好这是一个测试

ImportError: cannot import name 'punctuation' from 'text.symbols' (/root/autodl-tmp/data/python/vocal_clone/mb-istft-vits2-emotional/text/symbols.py)

In [63]:
['ZH','EN'].index('EN')

1

In [66]:
from prosody import TTSProsody

prosody = TTSProsody("models/bert-prosody/", "cuda")

text="小夕原本约好陪着表姐一起来相亲，表姐却因为胆小爽约"
prosody.char_model.tokenizer.tokenize(text)
char_embeds = prosody.get_char_embeds(text)
print(char_embeds)
# char_embeds = prosody.expand_for_phone(char_embeds, count_phone)

tensor([[-2.8755e-01, -7.3359e-01, -5.9503e-02,  ...,  5.3408e-02,
          9.7861e-01,  6.6443e-01],
        [ 7.6196e-01, -1.2703e+00,  9.0209e-02,  ..., -9.2874e-01,
          4.6115e-01,  2.6010e-01],
        [-1.8296e-01, -5.5231e-01, -3.2480e-01,  ...,  1.1862e+00,
          7.0646e-01,  6.0645e-01],
        ...,
        [ 5.6029e-01, -1.2006e+00,  2.8295e-01,  ..., -9.7576e-01,
          6.9785e-01,  2.2740e-01],
        [ 3.4936e-01, -1.6371e+00, -1.2956e-01,  ..., -1.2344e-03,
          5.7605e-01,  5.9578e-01],
        [-2.0375e-02, -1.3152e+00, -3.0811e-01,  ...,  3.6922e-01,
          7.4389e-01,  6.0087e-01]])


In [68]:
from text.mandarin import chinese_to_bopomofo

chinese_to_bopomofo(text)

Building prefix dict from the default dictionary ...


DEBUG:jieba:Building prefix dict from the default dictionary ...


Loading model from cache /tmp/jieba.cache


DEBUG:jieba:Loading model from cache /tmp/jieba.cache


Loading model cost 0.637 seconds.


DEBUG:jieba:Loading model cost 0.637 seconds.


Prefix dict has been built successfully.


DEBUG:jieba:Prefix dict has been built successfully.


'ㄒㄧㄠˇㄒㄧˉ ㄩㄢˊㄅㄣˇ ㄩㄝˉ ㄏㄠˇ ㄆㄟˊ ㄓㄜ˙ ㄅㄧㄠˇㄐㄧㄝˇ ㄧˉ ㄑㄧˇㄌㄞˊ ㄒㄧㄤˉㄑㄧㄣˉ， ㄅㄧㄠˇㄐㄧㄝˇ ㄑㄩㄝˋ ㄧㄣˉㄨㄟˋ ㄉㄢˇㄒㄧㄠˇ ㄕㄨㄤˇㄩㄝˉ'

In [12]:
from bert_extract import VITS_PinYin

text="小夕原本约好陪着表姐一起来相亲，表姐却因为胆小爽约"

py = VITS_PinYin("models/bert-prosody/", "cuda")
a = py.chinese_to_phonemes(text)
print(a.shape)

torch.Size([27, 256])
(51, 256)


ModuleNotFoundError: No module named 'audonnx'